# Case Study: Olympic medals

In this case study we explore the history of medals in the summer and winter olympics

The case study is divided into several parts:
- Goals
- Parsing
- Preparation (cleaning)
- Processing
- Exploration
- Visualization
- Conclusion

## Goals

In this section we define questions that will be our guideline througout the case study

- Which countries are over-/underperforming?
- Are some countries exceptional in some sports?
- Do physical traits have an influence on some sports?

We'll (try to) keep these question in mind when performing the case study.

## Parsing

we start out by importing all necessary libraries

In [1]:
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

in order to download datasets from kaggle, we need an API key to access their API, we'll make that here

In [2]:
if not os.path.exists("/root/.kaggle"):
    os.mkdir("/root/.kaggle")

with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(
        {
            "username":"lorenzf",
            "key":"7a44a9e99b27e796177d793a3d85b8cf"
        }
        , f)

now we can import kaggle too and download the datasets

In [3]:
import kaggle
kaggle.api.dataset_download_files(dataset='heesoo37/120-years-of-olympic-history-athletes-and-results', path='./data', unzip=True)

the csv files are now in the './data' folder, we can now read them using pandas, here is the list of all csv files in our folder

In [4]:
os.listdir('./data')

['noc_regions.csv', 'athlete_events.csv']


The file of our interest is 'athlete_events.csv', it contains every contestant in every sport since 1896. Let's print out the top 5 events.

In [5]:
athlete_events = pd.read_csv('./data/athlete_events.csv')
print('shape: ' + str(athlete_events.shape))
athlete_events.head()

shape: (271116, 15)


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


Seems we have a name, gender, age, height and weight of the contestant, as wel as the country they represent, the games they attended located in which city. The last 3 columns specify the sport, event within the sport and a possible medal. Presumably the keeping of their score would have been difficult as different sports use different score metrics which would be hard to compare.

In [6]:
noc_regions = pd.read_csv('./data/noc_regions.csv')
print('shape: ' + str(noc_regions.shape))
noc_regions.head()

shape: (230, 3)


,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


## Preparation

here we perform tasks to prepare the data in a more pleasing format.

### Data Types

Before we do anything with our data, it is good to see if our data types are in order

In [7]:
athlete_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [8]:
athlete_events[['Sex', 'Team', 'Season', 'City', 'Sport', 'Event']] = athlete_events[['Sex', 'Team', 'Season', 'City', 'Sport', 'Event']].astype('category')
athlete_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   ID      271116 non-null  int64   
 1   Name    271116 non-null  object  
 2   Sex     271116 non-null  category
 3   Age     261642 non-null  float64 
 4   Height  210945 non-null  float64 
 5   Weight  208241 non-null  float64 
 6   Team    271116 non-null  category
 7   NOC     271116 non-null  object  
 8   Games   271116 non-null  object  
 9   Year    271116 non-null  int64   
 10  Season  271116 non-null  category
 11  City    271116 non-null  category
 12  Sport   271116 non-null  category
 13  Event   271116 non-null  category
 14  Medal   39783 non-null   object  
dtypes: category(6), float64(3), int64(2), object(4)
memory usage: 20.8+ MB


### Missing values

for each dataframe we apply a few checks in order to see the quality of data

In [9]:
print(100*athlete_events.isna().sum()/athlete_events.shape[0])

ID         0.000000
Name       0.000000
Sex        0.000000
Age        3.494445
Height    22.193821
Weight    23.191180
Team       0.000000
NOC        0.000000
Games      0.000000
Year       0.000000
Season     0.000000
City       0.000000
Sport      0.000000
Event      0.000000
Medal     85.326207
dtype: float64


Age, 3.5% missing: 

Here we can't do much about it, we could impute using mean or median by looking at other contestants from the same sport/event, however I  have a feeling that missing ages might be prevalent in the same sports.


In [10]:
athlete_events.groupby('Year')['Age'].apply(lambda x: x.isna().sum()).sort_values(ascending=False).head(25)

Year
1948    1176
1924    1142
1928     963
1920     845
1900     790
1906     743
1908     649
1956     638
1932     330
1952     277
1904     274
1960     221
1984     216
1936     213
1980     187
1896     163
1912     156
1968     118
1988     110
1972      96
1964      56
1976      52
1992      44
1996       8
1994       2
Name: Age, dtype: int64

In [11]:
athlete_events.groupby('Sport')['Age'].apply(lambda x: x.isna().sum()).sort_values(ascending=False).head(25)

Sport
Gymnastics           1179
Athletics            1117
Shooting              821
Fencing               715
Cycling               678
Rowing                526
Swimming              524
Art Competitions      507
Wrestling             491
Football              375
Boxing                318
Sailing               285
Weightlifting         206
Hockey                204
Water Polo            200
Equestrianism         193
Basketball            186
Tennis                124
Diving                121
Archery                80
Alpine Skiing          78
Bobsleigh              72
Modern Pentathlon      53
Rugby                  48
Tug-Of-War             44
Name: Age, dtype: int64

Although some sports and years are more problematic, we cannot pinpoint a specific group where ages are missing. Imputing with mean or median would drasticly influence the distribution and standard deviation later on. I opt to leave the missing values as is and drop rows with NaN's when using age in calculations. 

Height & Weight, 22 & 23 % missing:

Similar to the Age, yet much more are missing, to a point where dropping would become problematic. Let's see if we can find a hotspot of missing data.

In [12]:
athlete_events.groupby('Year')[['Height', 'Weight']].apply(lambda x: x.isna().sum()).sort_values(by='Height', ascending=False).head(25)

,Height,Weight
Year,,
1952,7170,7171
1948,6311,6329
1936,6209,6414
1924,4719,5003
1928,4599,4856
1956,3748,3754
1920,3525,3821
1912,3319,3444
1992,3175,3157


In [13]:
athlete_events.groupby('Sport')[['Height', 'Weight']].apply(lambda x: x.isna().sum()).sort_values(by='Height', ascending=False).head(25)

,Height,Weight
Sport,,
Gymnastics,8045,8372
Athletics,5717,6023
Swimming,4045,4391
Shooting,3779,4148
Fencing,3773,4195
Art Competitions,3519,3523
Cycling,2883,3029
Rowing,2675,2662
Alpine Skiing,2435,2479


Again, no hotspots. For the same reason (distribution) we will not be imputing values, although for machine learning reasons this might be useful to increase the training pool. We will drop the rows with missing values whenever we use the height/weight columns. It would be wise here to inform our audience that conclusions on this data might be skewed by a possible bias - there might be a reason the data is missing - which might in turn cause us to make a wrongful conclusion!

Medal, 85% Missing:

Lastly we see that most are missing the medal, this is obviously that they did not win one. We could boldly assume that since each event has 3 medals, there must be an average of 20 contestants (17/20 = 85%). But this might be deviating over time and sport.

### Duplicates

For any reason, our dataset might be containing duplicates that would be counted twice and will introduce a bias we would not want. On the other hand, duplicates can be subjected to interpretation, here we would say that if 2 records share a name, gender, NOC, Games and event, the rows would be identical.
This would mean that the person would have performed twice in the same event for the same games under the same flag. The illustration below demonstrates a duplicate.

In [14]:
athlete_events[athlete_events.Name == 'Jacques Doucet']

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
57956,29661,Jacques Doucet,M,NaN,NaN,NaN,Favorite-17,FRA,1900 Summer,1900,Summer,Paris,Sailing,Sailing Mixed Open,NaN
57957,29661,Jacques Doucet,M,NaN,NaN,NaN,Favorite-1,FRA,1900 Summer,1900,Summer,Paris,Sailing,Sailing Mixed 2-3 Ton,Silver
57958,29661,Jacques Doucet,M,NaN,NaN,NaN,Favorite-1,FRA,1900 Summer,1900,Summer,Paris,Sailing,Sailing Mixed 2-3 Ton,Silver


We can se that Jacques for some reason is listed twice for the Sailing Mixed 2-3 Ton event. He won silver, but coming in second is no excused to be listed a second time! Perhaps we can find out where things went wrong by investigating in which year the duplicates appear.

In [15]:
duplicate_events = athlete_events[athlete_events.duplicated(['Name', 'Sex', 'NOC', 'Games', 'Event'])]
duplicate_events.groupby(['Year'])['Name'].count()

Year
1900    110
1908     35
1924    126
1928    347
1932    504
1936    258
1948    100
1968      2
1996      2
1998      3
2002      3
2012      1
Name: Name, dtype: int64

Seems most of them happen before 1948, perhaps due to errors in manual entries, it feels safe to delete them.

In [16]:
athlete_events = athlete_events.drop_duplicates(['Name', 'Sex', 'NOC', 'Games', 'Event'])

### Indexing

It is more convenient to work with an index, our dataset already contains an id which we can use as index

In [17]:
athlete_events = athlete_events.set_index('ID')
athlete_events.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
ID,,,,,,,,,,,,,,
1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## Processing

### Medals per country per sport
To find out which country (NOC) performs the best, we would like to have a dataframe with 3 columns ['Gold', 'Silver', 'Bronze'] containing the count of each, as row index, we would have the games and the NOC, thus a multiindex.
An important detail is that team sports are given multiple medals, as indicated by the exampe below. Be careful as bias might not always as visible.

In [18]:
athlete_events[(athlete_events.Event == "Basketball Men's Basketball")&(athlete_events.Games=='1992 Summer')&(athlete_events.Medal=='Gold')]

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
ID,,,,,,,,,,,,,,
7901,Charles Wade Barkley,M,29.0,198.0,114.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold
11668,Larry Joe Bird,M,35.0,205.0,100.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold
30009,Clyde Austin Drexler,M,30.0,200.0,101.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold
33553,Patrick Aloysius Ewing,M,29.0,213.0,109.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold
55424,"Earvin ""Magic"" Johnson, Jr.",M,32.0,205.0,100.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold
55881,Michael Jeffrey Jordan,M,29.0,198.0,90.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold
65809,Christian Donald Laettner,M,22.0,211.0,107.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold
74176,Karl Malone,M,29.0,205.0,116.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold
83179,"Christopher Paul ""Chris"" Mullin",M,28.0,200.0,98.0,United States,USA,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,Gold


The preprocessing for this dataframe seem complex but is combination of several operations:

- drop all records with no medals
- drop duplicates based on 'Games', 'NOC' , 'Event', 'Medal' to correct for team sports
- group per 'Games', 'NOC' , 'Medal'
- aggregate groups by calculating their size

At this point, we have a single column containing the amount of medals and 3 indices: 'Games' , 'NOC' and 'Medal'

- unstack the 'Medal' column to obtain 3 columns 'Gold', 'Silver', 'Bronze'
- make sure the order of columns is 'Gold', 'Silver', 'Bronze'
- drop rows where no medals are won, as we do not need those rows

This operation looks like the following:

In [19]:
medals_country_df = athlete_events.dropna(subset=['Medal']).drop_duplicates(['Games', 'NOC', 'Event']).groupby(['Games', 'NOC', 'Medal', 'Sport']).size().unstack('Medal')[['Gold', 'Silver', 'Bronze']]#.dropna(how='all')#.fillna(0)
medals_country_df = medals_country_df[medals_country_df.sum(axis='columns')>0]
medals_country_df

Medal                      Gold  Silver  Bronze
Games       NOC Sport                          
1896 Summer AUS Athletics     2       0       0
                Tennis        0       0       1
            AUT Cycling       1       0       2
                Swimming      1       1       0
            DEN Fencing       0       0       1
...                         ...     ...     ...
2016 Summer UZB Wrestling     0       0       3
            VEN Athletics     0       1       0
                Boxing        0       0       1
                Cycling       0       0       1
            VIE Shooting      1       1       0

[6915 rows x 3 columns]

### average statistics per year, country and sport

In [20]:
avg_stats_df = athlete_events.groupby(['Sex', 'NOC', 'Games', 'Sport'])[['Age', 'Height', 'Weight']].mean().dropna()
avg_stats_df

Age  Height  Weight
Sex NOC Games       Sport                          
F   AFG 2004 Summer Athletics  18.0   180.0    56.0
                    Judo       18.0   165.0    70.0
        2008 Summer Athletics  22.0   180.0    56.0
        2012 Summer Athletics  23.0   160.0    52.0
        2016 Summer Athletics  20.0   165.0    55.0
...                             ...     ...     ...
M   ZIM 2016 Summer Archery    37.0   186.0    78.0
                    Athletics  29.6   167.6    63.2
                    Rowing     27.0   191.0    87.0
                    Shooting   42.0   182.0    80.0
                    Swimming   22.0   181.0    84.0

[31329 rows x 3 columns]

## Exploration

## Visualization

## Summary